Before you begin, execute this cell to import numpy and packages from the D-Wave Ocean suite, and all necessary functions for the gate-model framework you are going to use, whether that is the Forest SDK or Qiskit. In the case of Forest SDK, it also starts the qvm and quilc servers.

In [1]:
%run -i "assignment_helper.py"
%matplotlib inline

/Users/gianlucafugante/anaconda3/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


Available frameworks:
Qiskit
D-Wave Ocean


**Exercise 1** (1 point). We want to solve the equation $Ax=b$ with $A = \begin{bmatrix}1 & 0 \\0 & -1 \\ \end{bmatrix}$ and $b =\begin{bmatrix} 0 \\ 1 \\ \end{bmatrix}$ with quantum matrix inversion. We will encode $A$ in the unitary matrix $U=e^{iAt_0}$ with $t_0=\pi/2$, and $b$ in a register. With the ancilla (qubit 0), the eigenvalue registers (or the ancilla qubits of phase estimation, qubits 1 and 2), and the eigenstate, you will need a total of four qubits and one classical register for post-selection. Prepare the superposition in the eigenvalue register and the vector $b$. Place your solution in an object called `hhl`.

In [40]:
###
### YOUR CODE HERE
###

qr = QuantumRegister(4)
cr = ClassicalRegister(4)

hhl = QuantumCircuit(qr, cr)

hhl.h(qr[1])
hhl.h(qr[2])
hhl.x(qr[3])

hhl.draw()


q271_0: ─────
        ┌───┐
q271_1: ┤ H ├
        ├───┤
q271_2: ┤ H ├
        ├───┤
q271_3: ┤ X ├
        └───┘
  c5: 4/═════

In [41]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j, 0. +0.j]))

**Exercise 2** (2 points). Start the quantum phase estimation by applying $C-U^{2^0}$ and $C-U^{2^1}$. Extend the circuit with the two appropriate gates.

In [42]:
###
### YOUR CODE HERE
###

hhl.cu1(2*np.pi, qr[1], qr[3])
hhl.cu1(np.pi, qr[2], qr[3])

hhl.draw()

q271_0: ──────────────────────
        ┌───┐                 
q271_1: ┤ H ├─■───────────────
        ├───┤ │               
q271_2: ┤ H ├─┼────────■──────
        ├───┤ │U1(2π)  │U1(π) 
q271_3: ┤ X ├─■────────■──────
        └───┘                 
  c5: 4/══════════════════════

In [43]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
                                          0. +0.j,  0. +0.j,  0.5+0.j,  0. +0.j,  0.5+0.j,  0. +0.j,
                                         -0.5+0.j,  0. +0.j, -0.5+0.j,  0. +0.j]))

**Exercise 3** (1 point). Apply the quantum inverse Fourier transformation. Don't forget the swap just before the transformation.

In [44]:
###
### YOUR CODE HERE
###

hhl.swap(qr[1], qr[2])
hhl.h(qr[2])
hhl.cu1(-np.pi / 2, qr[1], qr[2])
hhl.h(qr[1])

hhl.draw()

q271_0: ──────────────────────────────────────────────
        ┌───┐                                    ┌───┐
q271_1: ┤ H ├─■────────────────X───────■─────────┤ H ├
        ├───┤ │                │ ┌───┐ │U1(-π/2) └───┘
q271_2: ┤ H ├─┼────────■───────X─┤ H ├─■──────────────
        ├───┤ │U1(2π)  │U1(π)    └───┘                
q271_3: ┤ X ├─■────────■──────────────────────────────
        └───┘                                         
  c5: 4/══════════════════════════════════════════════

In [45]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
                                         0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]))

**Exercise 4** (1 point). After this step, swap the eigenvalue registers again (this time to perform the inversion) and apply the controlled rotation. Use the same angle as in the lecture notebook.

In [46]:
###
### YOUR CODE HERE
###

theta_0 = 0.392699
theta_1 = 0.19634955

hhl.swap(qr[1], qr[2])
hhl.cu3(theta_0, 0, 0, qr[1], qr[0])  # CRY0
hhl.cu3(theta_1, 0, 0, qr[2], qr[0]);  # CRY1

hhl.draw()

┌──────────────┐»
q271_0: ─────────────────────────────────────────────────┤ U3(1π/8,0,0) ├»
        ┌───┐                                    ┌───┐   └──────┬───────┘»
q271_1: ┤ H ├─■────────────────X───────■─────────┤ H ├─X────────■────────»
        ├───┤ │                │ ┌───┐ │U1(-π/2) └───┘ │                 »
q271_2: ┤ H ├─┼────────■───────X─┤ H ├─■───────────────X─────────────────»
        ├───┤ │U1(2π)  │U1(π)    └───┘                                   »
q271_3: ┤ X ├─■────────■─────────────────────────────────────────────────»
        └───┘                                                            »
  c5: 4/═════════════════════════════════════════════════════════════════»
                                                                         »
«        ┌──────────────┐
«q271_0: ┤ U3(π/16,0,0) ├
«        └──────┬───────┘
«q271_1: ───────┼────────
«               │        
«q271_2: ───────■────────
«                        
«q271_3: ────────────────
«                        
«  c5: 4/════════════════
«

In [47]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.99518473+0.j, 0.09801714+0.j, 0.        +0.j, 0.        +0.j]))

**Exercise 5** (2 points). Uncompute the eigenvalue register

In [48]:
###
### YOUR CODE HERE
###
hhl.swap(qr[1], qr[2])
hhl.h(qr[1])
hhl.cu1(np.pi / 2, qr[1], qr[2])
hhl.h(qr[2])
hhl.swap(qr[1], qr[2])
hhl.cz(qr[2], qr[3])
hhl.h(qr[1])
hhl.h(qr[2])

hhl.draw()

┌──────────────┐»
q271_0: ─────────────────────────────────────────────────┤ U3(1π/8,0,0) ├»
        ┌───┐                                    ┌───┐   └──────┬───────┘»
q271_1: ┤ H ├─■────────────────X───────■─────────┤ H ├─X────────■────────»
        ├───┤ │                │ ┌───┐ │U1(-π/2) └───┘ │                 »
q271_2: ┤ H ├─┼────────■───────X─┤ H ├─■───────────────X─────────────────»
        ├───┤ │U1(2π)  │U1(π)    └───┘                                   »
q271_3: ┤ X ├─■────────■─────────────────────────────────────────────────»
        └───┘                                                            »
  c5: 4/═════════════════════════════════════════════════════════════════»
                                                                         »
«        ┌──────────────┐                                    
«q271_0: ┤ U3(π/16,0,0) ├────────────────────────────────────
«        └──────┬───────┘   ┌───┐                  ┌───┐     
«q271_1: ───────┼─────────X─┤ H ├─■──────────────X─┤ H ├─────
«               │         │ └───┘ │U1(π/2) ┌───┐ │ └───┘┌───┐
«q271_2: ───────■─────────X───────■────────┤ H ├─X───■──┤ H ├
«                                          └───┘     │  └───┘
«q271_3: ────────────────────────────────────────────■───────
«                                                            
«  c5: 4/════════════════════════════════════════════════════
«

In [49]:
amplitudes

array([ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        1.99673462e-16+1.22464680e-16j,  0.00000000e+00+0.00000000e+00j,
       -9.00836673e-17+1.06030098e-16j, -1.79187517e-17+2.10906933e-17j,
        9.95184726e-01-3.04687447e-16j,  9.80171449e-02-3.00090957e-17j,
        8.78935449e-17+7.23348051e-17j,  2.66622119e-17+2.19425204e-17j])

In [50]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.99518473+0.j, 0.09801714+0.j, 0.        +0.j, 0.        +0.j,
                                         0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j]))

At this point, if we measure 1 in the ancilla (qubit 0), the state will be proportional to $A^{-1}b = \sum_j \beta_j\lambda_j^{-1}\left|u_j\right\rangle=0 \frac{1}{1} |0\rangle + 1 \frac{1}{-1} |1\rangle=-|1\rangle$.

In [51]:
hhl.measure(qr, cr)

backend = qiskit.BasicAer.get_backend('qasm_simulator') # the device to run on
result = execute(hhl, backend, shots=100).result()
counts  = result.get_counts(hhl)
print(counts)

{'1000': 100}
